In [1]:
import pandas as pd
import numpy as np
from data import clean_dataset
from features import Encoder

In [2]:
RAW_DATA_PATH = "../data/raw/olx_house_price_Q122.csv"

In [3]:
df = clean_dataset(RAW_DATA_PATH)
df.head()

,price,offer_type,floor,area,rooms,offer_type_of_building,market,longitude,latitude,city_name,population,voivodeship
0,240000.0,Private,1.0,27.00,1,Housing Block,primary,15.565104,51.263035,Bolesławiec,39603.0,Lower Silesia
1,250000.0,Private,1.0,35.00,1,Housing Block,primary,17.349964,51.039829,Jelcz-Laskowice,15828.0,Lower Silesia
2,259000.0,Estate Agency,2.0,25.00,1,Housing Block,primary,17.032669,51.108978,Wrocław,634487.0,Lower Silesia
3,269000.0,Private,3.0,26.18,1,Apartment Building,primary,17.032669,51.108978,Wrocław,634487.0,Lower Silesia
4,258000.0,Estate Agency,3.0,26.00,1,Housing Block,primary,17.032669,51.108978,Wrocław,634487.0,Lower Silesia


In [4]:
pd.DataFrame({"number": df.isnull().sum(), "percentage":df.isnull().sum().sort_values(ascending=False)/len(df)})

,number,percentage
price,0,0.0
offer_type,0,0.0
floor,0,0.0
area,0,0.0
rooms,0,0.0
offer_type_of_building,0,0.0
market,0,0.0
longitude,0,0.0
latitude,0,0.0
city_name,0,0.0


In [5]:
enc = Encoder()
enc.fit(df)

In [6]:
dict_f = {
    "price": 654.16,
    "offer_type": "Private",
    "floor": np.nan,
    "area": 230000.0,
    "rooms": 4,
    "offer_type_of_building": "Loft",
    "market": "primary",
    "longitude": 52.517,
    "latitude": 20.49848,
    "city_name": "Warszawa",
    "population": 2000000,
    "voivodeship": "Masovia",
}

to_p = enc.featurize(dict_f)

In [7]:
to_p

array([[ 5.100000e+02,  7.000000e+00,  0.000000e+00,  1.000000e+00,
         0.000000e+00,  0.000000e+00,  1.000000e+00,  0.000000e+00,
         0.000000e+00,  0.000000e+00,  1.000000e+00, -2.000000e+00,
         6.541600e+02,  2.300000e+05,  4.000000e+00,  5.251700e+01,
         2.049848e+01,  2.000000e+06]])